In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from pybit import inverse_perpetual  # <-- import HTTP & WSS for inverse perp
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from dotenv import load_dotenv
import os
import pandas as pd
import time
import ta 
import numpy as np
import warnings
warnings.simplefilter("ignore")

In [2]:
load_dotenv()

True

In [3]:
#Loading my Bybit's API keys from the dotenv file
api_key_pw = os.getenv('api_key_bot_IP')
api_secret_pw = os.getenv('api_secret_bot_IP')

In [4]:
#Stablishing Connection with the API (SPOT)
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)

#Creating a Dataframe of ETH real price (1m)
frame = pd.DataFrame(session_auth.query_kline(symbol="ETHUSDT", interval="1m")["result"])
frame

,0,1,2,3,4,5,6,7,8,9,10
0,1671763680000,1219.75,1220.18,1217.92,1219.15,52.32401,1671763740000,63776.5826515,187,0,0
1,1671763740000,1219.15,1220.17,1219.02,1220.17,142.56028,1671763800000,173917.257544,108,0,0
2,1671763800000,1220.17,1220.24,1219.81,1220.24,42.49579,1671763860000,51845.2955234,92,0,0
3,1671763860000,1220.24,1220.79,1220.23,1220.72,25.63202,1671763920000,31286.0827911,86,0,0
4,1671763920000,1220.72,1221.01,1220.45,1220.71,56.67538,1671763980000,69196.3045914,121,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,1671823380000,1218.89,1219.01,1218.71,1219.01,7.86436,1671823440000,9585.1089972,23,0,0
996,1671823440000,1219.01,1219.04,1218.77,1218.94,6.71812,1671823500000,8189.0664157,22,0,0
997,1671823500000,1218.94,1219.24,1218.88,1218.93,3.33446,1671823560000,4064.6620726,19,0,0
998,1671823560000,1218.93,1219.9,1218.93,1219.84,21.98283,1671823620000,26808.4324262,43,0,0


In [5]:
#This function gets Real BTC Price Data and creates a smooth dataframe that refreshes every 15 minutes
def get5minutedata():
    frame = pd.DataFrame(session_auth.query_kline(symbol="ETHUSDT", interval="5m")["result"])
    frame = frame.iloc[:,: 6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [6]:
df = get5minutedata()
df

,Open,High,Low,Close,Volume
Time,,,,,
2022-12-20 08:10:00,1209.00,1210.07,1207.13,1207.91,243.93021
2022-12-20 08:15:00,1207.91,1208.52,1205.86,1206.09,161.61670
2022-12-20 08:20:00,1206.09,1207.02,1205.62,1205.72,83.03039
2022-12-20 08:25:00,1205.72,1208.00,1205.72,1206.76,78.89439
2022-12-20 08:30:00,1206.76,1207.01,1205.88,1206.67,79.62236
...,...,...,...,...,...
2022-12-23 19:05:00,1218.04,1219.08,1217.92,1219.00,58.71017
2022-12-23 19:10:00,1219.00,1219.96,1218.87,1219.50,59.07250
2022-12-23 19:15:00,1219.50,1220.17,1219.29,1219.34,75.06097


In [59]:
#Function to apply some technical indicators from the ta library
def apply_technicals(df):
    df["K"] = ta.momentum.stochrsi(df.Close, window= 14)
    #df["D"] = df["K"].rolling(3).mean()
    df["RSI"] = ta.momentum.rsi(df.Close, window = 14)
    df.dropna(inplace=True)

In [60]:
apply_technicals(df)

In [61]:
class Signals:
    def __init__(self, df, lags):
        self.df = df
        self.lags = lags
    
    #Checking if we have a trigger in the last n time steps
    def get_trigger(self):
        df_2 = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df["RSI"].shift(i) > 75)
            df_2 = df_2.append(mask, ignore_index = True)
        return df_2.sum(axis= 0)
    
    # Is the trigger fulfilled and are all buying conditions fulfilled?
    def decide(self):
         self.df["trigger"] = np.where(self.get_trigger(), 1, 0)
         self.df["Sell"]= np.where((self.df.trigger), 1, 0)

In [62]:
inst = Signals(df, 1)

inst.decide()

In [63]:
df[df.Sell==1]

,Open,High,Low,Close,Volume,K,RSI,trigger,Sell
Time,,,,,,,,,
2022-12-17 23:05:00,1185.38,1189.97,1184.95,1186.60,378.23390,1.000000,75.655661,1,1
2022-12-17 23:10:00,1186.60,1186.79,1183.61,1184.43,175.99512,0.649459,64.603244,1,1
2022-12-18 04:10:00,1188.85,1193.92,1188.85,1193.23,477.25135,1.000000,84.405178,1,1
2022-12-18 04:15:00,1193.23,1194.09,1191.22,1191.55,152.26870,0.590217,72.203906,1,1
2022-12-18 04:20:00,1191.55,1196.48,1191.10,1194.00,743.34396,0.762942,77.346795,1,1
2022-12-18 04:25:00,1194.00,1195.76,1194.00,1194.86,242.09583,0.812677,78.827640,1,1
2022-12-18 04:30:00,1194.86,1196.40,1193.34,1193.78,236.56499,0.597633,72.424737,1,1
2022-12-18 18:00:00,1178.42,1191.00,1178.18,1186.86,805.39472,1.000000,82.659558,1,1
2022-12-18 18:05:00,1186.86,1186.95,1183.11,1184.29,280.38166,0.622141,69.542741,1,1


In [64]:
#The mail addresses and password
sender_address = 'pythontradingbot11@gmail.com'
sender_pass = os.getenv('mail_key')
receiver_address = os.getenv('mail')

message = MIMEMultipart() 
message_SL = MIMEMultipart()
message_TP = MIMEMultipart()
message_RSI = MIMEMultipart()
message_Others = MIMEMultipart()

In [65]:
def strategy_short(qty, open_position = False):
    df= get5minutedata()
    apply_technicals(df)
    inst = Signals(df, 1)
    inst.decide()
    print(f'Current Time is ' + str(df.index[-1]))
    print(f'Current Close is '+str(df.Close.iloc[-1]))
    print(f'Current RSI is ' + str(df.RSI.iloc[-1]))
    print("-----------------------------------------")

    if df.Sell.iloc[-1]:
        mail_content = "ETH Open Short"
        message.attach(MIMEText(mail_content, 'plain'))
        
        # Create SMTP session for sending the mail
        session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
        session_mail.starttls()  # enable security

        # login with mail_id and password
        session_mail.login(sender_address, sender_pass)
        text = message.as_string()
        session_mail.sendmail(sender_address, receiver_address, text)
        session_mail.quit()

        from pybit import usdt_perpetual
        session = usdt_perpetual.HTTP(
        endpoint='https://api.bybit.com',
        api_key = api_key_pw,
        api_secret= api_secret_pw)

        buyprice = round(df.Close.iloc[-1],2)

        print("-----------------------------------------")

        print(f"Buyprice: {buyprice}")

        print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

        order = session.place_active_order(symbol="ETHUSDT",
                                                side="Sell",
                                                order_type="Market",
                                                qty= qty,
                                                time_in_force="GoodTillCancel",
                                                reduce_only=False,
                                                close_on_trigger=False,
                                                take_profit = round(buyprice * 0.93,2),
                                                stop_loss = round(buyprice * 1.03,2))
        print(order)

        eth_order_id = str(order['result']['order_id'])
        print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
        print(f"Order id: {eth_order_id}") 
        print("---------------------------------------------------")

        open_position = True

    while open_position:
        time.sleep(30)
        from pybit import spot
        session_auth = spot.HTTP(
            endpoint='https://api.bybit.com',
            api_key = api_key_pw,
            api_secret= api_secret_pw)
            
        df = get5minutedata()
        apply_technicals(df)
        print(f"Buyprice: {buyprice}" + '             Close: ' + str(df.Close.iloc[-1]))
        print(f'Target: ' + str(round(buyprice * 0.93, 2)) + "                Stop: " + str(round(buyprice * 1.03, 2)))
        print(f'RSI Target: 53' + '                RSI: ' + str(df.RSI.iloc[-1]))
        print("---------------------------------------------------")

        if df.Close[-1] > buyprice* 1.03:
            print("Closed Position")
            open_position = False

            mail_content_SL = "ETH Short SL"
            message_SL.attach(MIMEText(mail_content_SL, 'plain'))

            # Create SMTP session for sending the mail
            session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
            session_mail.starttls()  # enable security

            # login with mail_id and password
            session_mail.login(sender_address, sender_pass)
            text = message_SL.as_string()
            session_mail.sendmail(sender_address, receiver_address, text)
            session_mail.quit()
            break

        elif df.Close[-1] < buyprice * 0.93: 
            print("Closed Position")
            open_position = False

            mail_content_TP = "ETH Short TP"
            message_TP.attach(MIMEText(mail_content_TP, 'plain'))

            # Create SMTP session for sending the mail
            session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
            session_mail.starttls()  # enable security
            
            # login with mail_id and password
            session_mail.login(sender_address, sender_pass)
            text = message_TP.as_string()
            session_mail.sendmail(sender_address, receiver_address, text)
            session_mail.quit()
            break

        elif df.RSI[-1] < 25:
            session = usdt_perpetual.HTTP(
            endpoint='https://api.bybit.com',
            api_key = api_key_pw,
            api_secret= api_secret_pw)

            try: 
                print(session.place_active_order(symbol="ETHUSDT",
                                                side="Buy",
                                                order_type="Market",
                                                qty= qty,
                                                time_in_force="GoodTillCancel",
                                                reduce_only=True,
                                                close_on_trigger=False))  

                print("---------------------------------------------------")
                print("Closed position")
                open_position = False
                mail_content_RSI = "ETH Short Closed - RSI < 25"
                message_RSI.attach(MIMEText(mail_content_RSI, 'plain'))

                # Create SMTP session for sending the mail
                session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
                session_mail.starttls()  # enable security

                # login with mail_id and password
                session_mail.login(sender_address, sender_pass)
                text = message_RSI.as_string()
                session_mail.sendmail(sender_address, receiver_address, text)
                session_mail.quit()
                break

            except: 
                print("Position already closed")
                open_position = False
                
                mail_content_Others = "Position Closed"
                message_Others.attach(MIMEText(mail_content_Others, 'plain'))

                # Create SMTP session for sending the mail
                session_mail = smtplib.SMTP('smtp.gmail.com', 587)  # use gmail with port
                session_mail.starttls()  # enable security

                # login with mail_id and password
                session_mail.login(sender_address, sender_pass)
                text = message_Others.as_string()
                session_mail.sendmail(sender_address, receiver_address, text)
                session_mail.quit()
                break



In [66]:
while True: 
    strategy_short(0.7)
    time.sleep(60)

Current Time is 2022-12-20 12:35:00
Current Close is 1213.15
Current RSI is 54.44682199118637
-----------------------------------------
Current Time is 2022-12-20 12:40:00
Current Close is 1215.72
Current RSI is 60.59012230118269
-----------------------------------------
Current Time is 2022-12-20 12:40:00
Current Close is 1215.84
Current RSI is 60.836868276238164
-----------------------------------------


In [ ]:
#ta.volume.VolumeWeightedAveragePrice(high: pandas.core.series.Series, low: pandas.core.series.Series, close: pandas.core.series.Series, volume: pandas.core.series.Series, window: int = 14, fillna: bool = False)